<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/contaminacionMadrid/preprocesadoDatosContaminacionMadrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Visualizacion de datos de contaminacion de Madrid. Hasta ahora dispongo de 

## Plaza del Cármen, NO2
## Casa de campo, NO2, O3
## Plaza elíptica, Fernández Ladreda,  NO2

Cuatro datasets

El formato es diario partiendo del día 1-1-2010 y las unidades mg/m^3

In [21]:
from google.colab import drive
from csv import reader

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# File IDs
Est56FL_CodPar08NO2_valuesonly='1swJzeitSbkv0RduzJToM8IBzQ5TOjs7F'
Est24CC_CodPar08NO2_valuesonly='1dOHl9CGJPtnN8bHbn7gEFLkodjYbcHAT'
Est03PlC_CodPar08NO2_valuesonly='10U2QEnAn2KTll8qWq-HVaINPGU3exrPq'
Est03PlC_CodPar14O3_valuesonly='1OQwSUPSqoR5C5Z76GflAxMYUnzufnfjv'


est56NO2 = drive.CreateFile({'id': Est56FL_CodPar08NO2_valuesonly})
est56NO2 = drive.CreateFile({'id': Est24CC_CodPar08NO2_valuesonly})
est56NO2 = drive.CreateFile({'id': Est03PlC_CodPar08NO2_valuesonly})
est56NO3 = drive.CreateFile({'id': Est03PlC_CodPar14O3_valuesonly})

print(list(reader(est56NO2.GetContentString())))

est56NO2_df = pd.read_csv(reader(est56NO2.GetContentString()), header=0)
est24NO2_df = pd.read_csv(reader(est56NO2.GetContentString()), header=0)
est03NO2_df = pd.read_csv(reader(est56NO2.GetContentString()), header=0)
est03O3_df  = pd.read_csv(reader(est56NO3.GetContentString()), header=0)



[['x'], [], ['2'], ['4'], ['.'], ['3'], ['2'], [], ['4'], ['5'], ['.'], ['8'], [], ['5'], ['9'], ['.'], ['1'], ['7'], [], ['6'], ['2'], ['.'], ['9'], ['6'], [], ['5'], ['4'], ['.'], ['3'], ['4'], [], ['3'], ['4'], ['.'], ['9'], ['4'], [], ['4'], ['3'], ['.'], ['2'], ['1'], [], ['3'], ['9'], ['.'], ['3'], ['1'], [], ['3'], ['9'], ['.'], ['9'], ['7'], [], ['4'], ['7'], ['.'], ['3'], ['4'], [], ['6'], ['7'], ['.'], ['5'], ['3'], [], ['6'], ['9'], ['.'], ['3'], ['9'], [], ['6'], ['8'], ['.'], ['4'], ['2'], [], ['3'], ['8'], ['.'], ['4'], [], ['7'], ['4'], ['.'], ['3'], ['6'], [], ['6'], ['5'], ['.'], ['6'], ['3'], [], ['5'], ['6'], ['.'], ['2'], ['1'], [], ['6'], ['1'], ['.'], ['5'], [], ['5'], ['6'], ['.'], ['5'], ['8'], [], ['4'], ['3'], ['.'], ['9'], ['3'], [], ['7'], ['9'], ['.'], ['8'], ['4'], [], ['9'], ['2'], ['.'], ['5'], ['8'], [], ['6'], ['7'], ['.'], ['9'], ['1'], [], ['5'], ['2'], ['.'], ['2'], [], ['4'], ['3'], ['.'], ['5'], ['7'], [], ['4'], ['4'], ['.'], ['9'], ['8'], [], ['

ValueError: ignored

In [52]:
from google.colab import drive
drive.mount('/content/drive/')

# Exploramos los directorios de drive. Cambiamos directorio a donde estan los datos
%cd /content/drive/My\ Drive/datosContaminacionMadrid

# Lo pasamos a dataframes
est56NO2_df = pd.read_csv('Est56FL_CodPar08NO2_valuesonly.txt', header=0)
est24NO2_df = pd.read_csv('Est24CC_CodPar08NO2_valuesonly.txt', header=0)
est03NO2_df = pd.read_csv('Est03PlC_CodPar08NO2_valuesonly.txt', header=0)
est03O3_df  = pd.read_csv('Est03PlC_CodPar14O3_valuesonly.txt', header=0)

print(est56NO2_df.iloc[0])
print(est56NO2_df.iloc[5])
print(est56NO2_df.iloc[6])
print(est56NO2_df.iloc[7])
print(est56NO2_df.iloc[8])

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/datosContaminacionMadrid
x    0.0
Name: 0, dtype: float64
x    0.0
Name: 5, dtype: float64
x    0.0
Name: 6, dtype: float64
x    0.0
Name: 7, dtype: float64
x    0.0
Name: 8, dtype: float64
